In [2]:
import os
os.environ["ZARR_V3_EXPERIMENTAL_API"] = "1"

In [1]:
from odc import stac as odc_stac
from pystac_client import Client
import xarray as xr
from pyproj import Transformer
import math
import xarray as xr
import numpy as np 
from rasterio.windows import from_bounds

Connect to eodc stac and open the collection for the Copernicus Land Monitoring data

In [ ]:
eodc_catalog = Client.open(
    "https://dev.stac.eodc.eu/api/v1",
)

pc_client=Client.open("https://dev.stac.eodc.eu/api/v1")
col = pc_client.get_collection("clms")
col

<CollectionClient id=clms>

Print the assets and select your desired dataset

In [7]:
print(list(col.assets.keys()))
asset = col.assets["VPP"]
asset.href, asset.media_type, asset.extra_fields

['ST', 'VPP']


('https://data.eodc.eu/collections/CLMS/CLMS.zarr',
 'application/vnd.zarr; version=3',
 {'bands': [{'name': 'AMPL', 'description': 'Season Amplitude'},
   {'name': 'EOSD', 'description': 'End-of-season Date'},
   {'name': 'EOSV', 'description': 'End-of-season Value'},
   {'name': 'LENGTH', 'description': 'Season Length'},
   {'name': 'LSLOPE', 'description': 'Slope of the Green-up Period'},
   {'name': 'MAXV', 'description': 'Season Maximum Value'},
   {'name': 'MINV', 'description': 'Season Minimum Value'},
   {'name': 'QFLAG', 'description': 'Quality Flag'},
   {'name': 'RSLOPE', 'description': 'Slope of the Green-down Period'},
   {'name': 'SOSD', 'description': 'Start-of-season Date'},
   {'name': 'SOSV', 'description': 'Start-of-season Value'},
   {'name': 'SPROD', 'description': 'Seasonal Productivity'},
   {'name': 'TPROD', 'description': 'Total Productivity'}],
  'zarr:group': 'VPP',
  'zarr:node_type': 'group',
  'zarr:zarr_format': 3})

In [9]:
store_url = asset.href
group_path = asset.extra_fields.get("zarr:group")  # may be None

print("href:", store_url)
print("group:", group_path)

href: https://data.eodc.eu/collections/CLMS/CLMS.zarr
group: VPP


Open the zarr file

In [10]:
coords = xr.open_zarr(
    store_url,
    group=group_path,
    zarr_version=3,
)

C:\Users\Franziska\AppData\Local\Temp\ipykernel_19144\3665847098.py:1: FutureWarning: zarr_version is deprecated, use zarr_format
  coords = xr.open_zarr(


Select the desired group and the season (1 or 2)

In [11]:
maxv_ds = xr.open_zarr(
    store_url,
    group="VPP/MAXV",
    zarr_version=3,
    chunks="auto",
)

C:\Users\Franziska\AppData\Local\Temp\ipykernel_19144\2837227757.py:1: FutureWarning: zarr_version is deprecated, use zarr_format
  maxv_ds = xr.open_zarr(


In [12]:

maxv_zarr = maxv_ds["SEASON1"].assign_coords(
    time=("time", coords["time"].values),
    x=("x", coords["x"].values),
    y=("y", coords["y"].values),
)

Select an area: Here Rosalia

The data is in EPSG3035, so we need to transform the corrdinates.

In [13]:
minlon, minlat, maxlon, maxlat = 16.25, 46.68, 16.31, 47.72
year = 2022

transformer = Transformer.from_crs(
    "EPSG:4326",
    "EPSG:3035",
    always_xy=True
)


def to_3035(minlon, minlat, maxlon, maxlat):
    x0, y0 = transformer.transform(minlon, minlat)
    x1, y1 = transformer.transform(maxlon, minlat)
    x2, y2 = transformer.transform(maxlon, maxlat)
    x3, y3 = transformer.transform(minlon, maxlat)

    minx_3035 = int(math.floor(min(x0, x1, x2, x3)))
    maxx_3035 = int(math.ceil (max(x0, x1, x2, x3)))
    miny_3035 = int(math.floor(min(y0, y1, y2, y3)))
    maxy_3035 = int(math.ceil (max(y0, y1, y2, y3)))

    return minx_3035, miny_3035, maxx_3035, maxy_3035

minx, miny, maxx, maxy = to_3035(minlon, minlat, maxlon, maxlat)

print(minx, miny, maxx, maxy)



4789576 2638650 4803578 2754238


Get the data in the selected area

In [14]:
subset = maxv_zarr.sel(
    time=year,
    x=slice(minx, maxx),
    y=slice(miny, maxy),
)

print(subset.values)

[[0.6665 0.6759 1.0142 ... 1.61   1.554  1.7099]
 [0.5018 1.0029 1.3698 ... 2.0067 1.8184 1.4919]
 [0.7442 1.3635 1.6927 ... 2.1152 2.3205 1.9617]
 ...
 [1.4486 2.0843 2.2482 ... 1.2622 1.2122 1.3095]
 [1.3237 1.3887 1.3536 ... 1.2472 1.0891 0.9634]
 [1.1522 1.0731 0.9404 ... 0.9311 0.806  0.6154]]
